In [79]:
from openbb import obb
import pandas as pd
import numpy as np
from scipy.stats import norm
import datetime as dt

In [89]:
def FetchRates(Start_Date=None, End_Date=None):
    treasury_data = obb.fixedincome.government.treasury_rates(start_date=Start_Date, end_date=End_Date, provider="federal_reserve").to_df()
    fed_funds = obb.fixedincome.rate.effr(start_date=Start_Date, end_date=End_Date, provider="federal_reserve").to_df()[['rate']].rename(columns={'rate': 'FedFunds'})
    sofr_data = obb.fixedincome.rate.sofr(start_date=Start_Date, end_date=End_Date, provider="federal_reserve").to_df()[['rate']].rename(columns={'rate': 'SOFR'})

    #Merge
    rates_data = treasury_data.join([fed_funds, sofr_data], how='outer')
    #Reset Index
    rates_data = rates_data.rename_axis('Date').reset_index()

    #Renaming Columns
    rates_data.rename(columns={
        "month_1": "1Mo", "month_2": "2Mo", "month_3": "3Mo", "month_6": "6Mo", 
        "year_1": "1Yr", "year_2": "2Yr", "year_3": "3Yr", 
        "year_5": "5Yr", "year_7": "7Yr", "year_10": "10Yr", "year_30": "30Yr"
    }, inplace=True)

    # Filter and Sort
    cols = ['Date', 'FedFunds', 'SOFR', '1Mo', '3Mo', '6Mo', '1Yr', '2Yr', '3Yr', '5Yr', '7Yr', '10Yr', '30Yr']
    rates_data = rates_data[cols].sort_values('Date')

    # Removing rows where primary benchmarks are missing (weekends/holidays)
    rates_data = rates_data.dropna(subset=['FedFunds', '1Mo'])
    
    # Forward Fill SOFR (Modern syntax)
    rates_data['SOFR'] = rates_data['SOFR'].ffill()
    
    return rates_data

In [12]:
# Filter for rows that contain any NaN values
#null_rows = rates_data[rates_data.isnull().any(axis=1)]
#null_rows

In [93]:
start_date = "2020-01-02"
end_date = dt.datetime.today().strftime('%Y-%m-%d')

rates_data=FetchRates(start_date, end_date)

In [ ]:
#Check for nulls
null_rows = rates_data[rates_data.isnull().any(axis=1)]
null_rows

,Date,FedFunds,SOFR,1Mo,3Mo,6Mo,1Yr,2Yr,3Yr,5Yr,7Yr,10Yr,30Yr
